In [24]:
## This program transforms fMRI subject data to NLP input-able feature data
##
## 1. Assume import subject data is at the root directory of this program.
## 2. To run this program, setup the {file directory} and {output directory} in your local harddrive.
## 3. ultimate output file is named as "word_feature_gauss_region_XX.txt" which XX indicates the region number
## 4. output file is delimited by ~ between voxels since other common charaters are already included in story context
## 5. depends on your model program, you may need to convert output file to other format.

import numpy as np
import pandas as pd
import scipy.signal as sg
from scipy.io import loadmat

In [25]:
def fMRI_initiate(file_name):
    subject = loadmat(file_name)
    time = subject['time']
    data = subject['data']
    meta = subject['meta']
    words = subject['words']
    
    result = {
        'time': time,
        'data': data,
        'meta': meta,
        'words': words
    }
    
    return result


In [26]:
def fMRI_data_by_region(data, meta, output_to_file = False):
    ## subject_data
    ## subject_meta
    meta['colToROInum'][0,0][0].shape[0]
    ROI_map = {}
    for index, x in np.ndenumerate(meta['colToROInum'][0,0][0]):
        if str(x) in ROI_map:
            ROI_map.get(str(x)).append(index[0])
        else:
            ROI_map[str(x)] = [index[0]]
    file_paths = []
    region_nums = []
    file_directory = "{file directory}" ##setup own directory
    file_prefix = "raw_region_"
    file_format = ".txt"
    for key in ROI_map.keys():
        region_data = data[:, ROI_map[str(key)]]
        assert(region_data.shape[1] == len(ROI_map[str(key)]))
        
        if output_to_file == True:
            out_path = file_directory + file_prefix + str(key) + file_format
            region_nums.append(key)
            np.savetxt(out_path, region_data,delimiter='~',fmt="%s")
    
    result = {
        'ROI_map': ROI_map,
        'file_directory': file_directory,
        'file_prefix': file_prefix,
        'file_format': file_format,
        'region_nums': region_nums
    }
    return result

In [27]:
## get words and its length, start time and duration
def get_words_start_time(words):
    count_records = words.shape[1]
    array_records = [];
    for i in range(count_records):
        single_row = words[0,i]
        transfer_row = np.array(["[" + single_row[0][0,0][0] + "]", len(single_row[0][0,0][0]), single_row[1][0,0], single_row[2][0,0]])
        array_records.append(transfer_row)
    return  np.array(array_records)

In [28]:
## fourfold copy data and expand time from 0,2 --> 0,0.5,1,1.5,2
def get_data_time_fourfold(data, time):
    subject_data_time =  np.concatenate((time, data), axis=1)
    subject_data_time = subject_data_time.astype(np.object)
    subject_data_time_fourfold = []
    for i in range(subject_data_time.shape[0]):
        temp = subject_data_time[i,:].copy()
        row_05 = temp.copy()
        row_05[0] += 0.5
        row_10 = temp.copy()
        row_10[0] += 1
        row_15 = temp.copy()
        row_15[0] += 1.5
        subject_data_time_fourfold.append(temp)
        subject_data_time_fourfold.append(row_05)
        subject_data_time_fourfold.append(row_10)
        subject_data_time_fourfold.append(row_15)
    subject_data_time_fourfold = np.array(subject_data_time_fourfold)
    
    for i in range(subject_data_time_fourfold.shape[0]-1):
        assert(subject_data_time_fourfold[i,0] == subject_data_time_fourfold[i+1, 0]-0.5)
    
    subject_data_time_fourfold_time = subject_data_time_fourfold[:,0:2] ## time - fourfold 0,0.5,1,1.5,2 ... 
    subject_data_time_fourfold_data = subject_data_time_fourfold[:,2:]  ## data - fourfold.
    
    assert(subject_data_time_fourfold_time.shape[0] == time.shape[0] * 4)
    assert(subject_data_time_fourfold_data.shape[0] == data.shape[0] * 4)
    assert(subject_data_time_fourfold_data.shape[1] == data.shape[1])
    
    result = {
        'fourfold_time': subject_data_time_fourfold_time,
        'fourfold_data': subject_data_time_fourfold_data
    }
    return result

In [29]:
## normalize fourfold data to be between 0 and 1 and round to certain decimals
def data_fourfold_normalize_round(data, dmax, dmin, round_scale=5):
    result = (data - dmin) / (dmax - dmin)
    result_round = np.around(result.astype(np.double), decimals=round_scale)
    return result_round

In [30]:
def merge_word_time_data(words, time, data):
    ## words: word_start_time
    ## time: fourfold_time,
    ## data: fourfold_data_normalized
    assert(time.shape[0] == data.shape[0])
    words.astype(np.object)
    time_data = np.concatenate((time, data), axis=1)
    result = []
    i = 0 # pointer for words [time has gap]
    j = 0 # pointer for time_data [time has no gap]
    while i < len(words) and j < len(time_data):
        if float(words[i,2]) == float(time_data[j,0]):
            word_time_data = np.append(words[i], time_data[j])
            i += 1
            j += 1
        elif float(words[i,2]) > float(time_data[j,0]):
            word_time_data = np.append(np.zeros(words.shape[1]), time_data[j])
            j += 1
        else:
            print('Code goes to unexpected block for i: ' + str(i) + ", j: "+str(j))
            word_time_data = np.append(words[i], np.zeros(time_data[j]))
            i += 1
        #result[j] = word_time_data
        result.append(word_time_data)
    while j < len(time_data):
        word_time_data = np.append(np.zeros(words.shape[1]), time_data[j])
        j += 1
        result.append(word_time_data)
        
    result = np.array(result)
    #assert(result.shape[0] == time_data.shape[0])
    #assert(result.shape[1] == words.shape[1] + time_data.shape[1])

    return result
    

In [31]:
def apply_gauss_window(data_start_col, data, window_size, delay = 0, round_scale=4):
    ## apply gaussian weights to data, assume data file has no "header"
    weights = sg.gaussian(window_size,1); ## 1 = deviation
    
    meta = data[:,0:data_start_col]
    pure_data = data[:,data_start_col:].astype(np.float)
    
    result = []
    for row in range(pure_data.shape[0]-window_size-1):
        weight_sum = np.zeros(pure_data.shape[1])
        for i in range(window_size):
            weight_sum = weight_sum + pure_data[row+delay+i] * weights[i]
        weight_avg = weight_sum / window_size 
        result.append(weight_avg)
    result = np.array(result)
    assert(result.shape[0] == data.shape[0] - window_size -1)
    assert(result.shape[1] == pure_data.shape[1])
    
    result = np.round(result, round_scale)
    output_result = np.concatenate((meta[0:result.shape[0]], result), axis = 1)
    return output_result

In [32]:
def build_header_for_region_features(word_feature_data, dataShape, region_num):
    ## this function only add header for particular region word features
    meta_col = ['token', 'length', 'start_time', 'duration', 'start_time2', 'section']
    assert(len(meta_col) + dataShape == word_feature_data.shape[1])
    for i in range(dataShape):
        meta_col.append(str(region_num) + '_v' + str(i+1))
    assert(len(meta_col) == word_feature_data.shape[1])
    result = np.concatenate((np.array(meta_col).reshape((1, word_feature_data.shape[1])), word_feature_data), axis=0)
    return result

In [35]:
def fMRI_process(file, gauss_window_size = 12):
    ## umbrella method to call all methods
    subject = fMRI_initiate(file)
    subject_words = subject['words']
    subject_time = subject['time']
    subject_data = subject['data']
    subject_meta = subject['meta']
    
    dmax, dmin = subject_data.max(), subject_data.min()
    print("File name: " + str(file))
    print("max value : " + str(dmax) + "; min value: " + str(dmin))
    print('gauss weights for window size : '+ str(gauss_window_size) + ' is: ' + str(sg.gaussian(gauss_window_size,1)))
    
    ## get words with start time and duration
    words_start_time = get_words_start_time(subject_words)
    
    #split raw data by region and write to harddrive
    region_info = fMRI_data_by_region(subject_data, subject_meta, output_to_file = True)
    region_nums = region_info['region_nums']
    region_ROI_map = region_info['ROI_map']
    region_file_directory = region_info['file_directory']
    region_file_prefix = region_info['file_prefix']
    region_file_format = region_info['file_format']
          
    #load raw data by region
    for region_num in region_nums:
        input_file = region_file_directory + region_file_prefix + str(region_num) + region_file_format
        region_data = np.loadtxt(open(input_file, "rb"), delimiter="~")
        assert(region_data.shape[0] == subject_data.shape[0])
        assert(region_data.shape[1] == len(region_ROI_map.get(str(region_num))))
        
        fourfold = get_data_time_fourfold(region_data, subject_time) ## fourfold
        
        fourfold_time = fourfold['fourfold_time']
        fourfold_data = fourfold['fourfold_data']
        
        del fourfold
        
        fourfold_data_norm = data_fourfold_normalize_round(fourfold_data, dmax, dmin, round_scale=6)
        
        raw_merge_result = merge_word_time_data(words_start_time, fourfold_time, fourfold_data)
        fine_merge_result = merge_word_time_data(words_start_time, fourfold_time, fourfold_data_norm)
        
        ## word feature for raw data
        raw_out_file = "{output directory}/word_feature_raw_region_" + str(region_num) + ".txt"
        raw_merge_result_with_header = build_header_for_region_features(raw_merge_result, fourfold_data.shape[1], region_num)
        np.savetxt(raw_out_file, raw_merge_result_with_header, delimiter='~',fmt="%s")
        del raw_merge_result_with_header
        del raw_merge_result
        
        ## word feature for normalized data
        fine_out_file = "{output directory}/word_feature_norm_region_" + str(region_num) + ".txt"
        fine_merge_result_with_header = build_header_for_region_features(fine_merge_result, fourfold_data.shape[1], region_num)
        np.savetxt(fine_out_file, fine_merge_result_with_header, delimiter='~',fmt="%s")
        del fine_merge_result_with_header
        
        ## word feature with Gaussian Window
        gauss_out_file = "{output directory}/word_feature_gauss_region_" + str(region_num) + ".txt"
        data_start_index = words_start_time.shape[1] + fourfold_time.shape[1]
        gauss_merge_result = apply_gauss_window(data_start_index, fine_merge_result, window_size = gauss_window_size, delay = 0, round_scale = 4)
        gauss_merge_result_with_header = build_header_for_region_features(gauss_merge_result, fourfold_data.shape[1], region_num)
        np.savetxt(gauss_out_file, gauss_merge_result_with_header, delimiter='~',fmt="%s")
        
    print('Done processing fMRI data')

In [34]:
fMRI_process(file = 'subject_8.mat', gauss_window_size = 12)

File name: subject_8.mat
max value : 1140; min value: -81
gauss weights for window size : 12 is: [2.69957850e-07 4.00652974e-05 2.18749112e-03 4.39369336e-02
 3.24652467e-01 8.82496903e-01 8.82496903e-01 3.24652467e-01
 4.39369336e-02 2.18749112e-03 4.00652974e-05 2.69957850e-07]
Done processing fMRI data
